In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

# Import necessary files
data1 = pd.read_excel('file1.xlsx')
data3 = pd.read_excel('file3.xlsx')
data4 = pd.read_excel('export-4d56c4d.xlsx')

data1['Name'] = data1['Company Name']
data1['Domain'] = data1['Website'].apply(lambda x: x.replace('https://www.', ''))

# Merge data and create final dataset
merged_data = pd.merge(data4, data1[['Domain', 'Rank']], on='Domain', how='left')
final_data = pd.concat([merged_data, data3], ignore_index=True)

# Select relevant columns
columns = ['Employee Estimate', 'Employees on Professional Networks', 'Employee Growth (Monthly)', 
           'Employee Growth (Quarterly)', 'Employee Growth (6 months)', 'Employee Growth (Annual)', 
           'Headquarters', 'Year Founded', 'Last Funding Amount', 'Last Funding Date', 
           'Total Funding Amount', 'Total Funding Rounds', 'Business Model', 'Last Touch Date', 
           'Last Pipeline Decline Date', 'Rank']
df = final_data[columns]

# US location check
us_states = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']

def is_us_location(location):
    if pd.isna(location):
        return 0.5
    return int(any(state in location for state in us_states))

df['is_us'] = df['Headquarters'].apply(is_us_location)

# Calculate company age
current_year = datetime.now().year
df['company_age'] = current_year - df['Year Founded']

# Check for tech-enabled sectors
tech_enabled_sectors = ['Software', 'Software Enabled', 'Tech', 'Technology']

def is_tech_enabled(sector):
    if pd.isna(sector):
        return 0
    return int(sector in tech_enabled_sectors)

df['is_tech_enabled'] = df['Business Model'].apply(is_tech_enabled)

df['Rank'] -= 1

# Clean data
df_cleaned = df.drop(columns=['Year Founded', 'Headquarters', 'Business Model'])
df_cleaned = df_cleaned.dropna(subset=['Rank'])

# Convert 'Employee Growth' columns to float
growth_columns = ['Employee Growth (Monthly)', 'Employee Growth (Quarterly)', 'Employee Growth (6 months)', 'Employee Growth (Annual)']
for col in growth_columns:
    df_cleaned[col] = df_cleaned[col].str.replace('%', '').astype(float)

# Process date columns
today = pd.Timestamp.now().tz_localize(None)

def safe_to_timestamp(x):
    if pd.isna(x):
        return np.nan
    try:
        return pd.to_datetime(x).tz_localize(None)
    except:
        return np.nan

date_columns = ['Last Funding Date', 'Last Touch Date', 'Last Pipeline Decline Date']
for col in date_columns:
    df_cleaned[col] = df_cleaned[col].apply(safe_to_timestamp)
    df_cleaned[f'Days Since {col}'] = (today - df_cleaned[col]).dt.days.replace({pd.NaT: np.nan})

# Drop original date columns
df_cleaned = df_cleaned.drop(columns=date_columns)

# Save the cleaned data to a CSV file
df_cleaned.to_csv('train_data_cb.csv', index=False)

/var/folders/jz/27btwlsn5hj2km5hc3rg4vfr0000gn/T/ipykernel_4447/1352961718.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['is_us'] = df['Headquarters'].apply(is_us_location)
/var/folders/jz/27btwlsn5hj2km5hc3rg4vfr0000gn/T/ipykernel_4447/1352961718.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['company_age'] = current_year - df['Year Founded']
/var/folders/jz/27btwlsn5hj2km5hc3rg4vfr0000gn/T/ipykernel_4447/1352961718.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of